# 🧬 Omics AI Explorer - Quick Start

Simple demo using the CoLoRS collection on HiFi Solves.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mfiume/omics-ai-python-library/blob/main/Omics_AI_Explorer_Quick_Start.ipynb)

In [ ]:
# Install and import
!pip install git+https://github.com/mfiume/omics-ai-python-library.git --quiet
from omics_ai import list_collections, list_tables, get_schema_fields, query
print("Ready!")

In [ ]:
# List collections on HiFi Solves
collections = list_collections("hifisolves")
print(f"Found {len(collections)} collections")
print(f"First: {collections[0]['name']}")

In [ ]:
# Explore CoLoRS collection
collection = "consortium-of-long-read-sequencing-colors"
tables = list_tables("hifisolves", collection)
print(f"Tables in CoLoRS: {len(tables)}")
for table in tables:
    print(f"- {table['display_name']}")

In [ ]:
# Query variants data
table = "collections.consortium_of_long_read_sequencing_colors.colors_small_variants"
result = query("hifisolves", collection, table, limit=10)

data = result['data']
print(f"Retrieved {len(data)} variants")

# Display as table
import pandas as pd
df = pd.DataFrame(data)
print(f"\nVariants table ({df.shape[0]} rows, {df.shape[1]} columns):")
display(df.head(10))

In [ ]:
# Query with filters - chromosome 1 only
filters = {
    "chrom": [{
        "operation": "EQ",
        "value": "chr1", 
        "type": "STRING"
    }]
}

result_filtered = query("hifisolves", collection, table, filters=filters, limit=10)
data_filtered = result_filtered['data']
print(f"Retrieved {len(data_filtered)} variants from chromosome 1")

# Display filtered results as table
if data_filtered:
    df_filtered = pd.DataFrame(data_filtered)
    print(f"\nChromosome 1 variants ({df_filtered.shape[0]} rows, {df_filtered.shape[1]} columns):")
    display(df_filtered.head(10))
else:
    print("No variants found on chromosome 1")

# Check other networks
for network in ["viral", "neuroscience", "biomedical"]:
    try:
        collections = list_collections(network)
        print(f"{network}: {len(collections)} collections")
    except Exception as e:
        print(f"{network}: error")